In [99]:
!pip install implicit
!pip install surprise
!pip install tensorflow
!pip install scikit-optimize
!pip install scipy
!pip install pandas
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from implicit import bpr
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split
import tensorflow as tf
import numpy as np

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [100]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [101]:
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

print(len(allHours))

174999


In [102]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated

reviewsPerUser = defaultdict(list) # Maps a user to their reviews

for l in allHours:
    user,item,review = l[0], l[1], l[2]
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    reviewsPerUser[user].append(review)

games = set()
for l in allHours:
    games.add(l[1])
gamesList = list(games)

## Would Play BPR

In [40]:
augValidSet = []
added = set()

for l in hoursValid:
    user, game, review = l[0], l[1], l[2]
    augValidSet.append((user, game, 1))
    added.add((user, game, 1))
    
    randGame = random.sample(gamesList, 1)[0]
    while randGame in itemsPerUser[user] or (user, randGame, 0) in added:
        randGame = random.sample(gamesList, 1)[0]
    
    augValidSet.append((user, randGame, 0))
    added.add((user, randGame, 0))

print(len(hoursValid))
print(len(augValidSet))
print(augValidSet[:3])

9999
19998
[('u00914251', 'g61913894', 1), ('u00914251', 'g56396594', 0), ('u44824365', 'g16586637', 1)]


In [41]:
userIDs,gameIDs = {},{}
interactions = []
userIDlookup, gameIDlookup = {}, {}
games = list(games)

for u, g, r in allHours:
    if not u in userIDs: 
        userIDs[u] = len(userIDs)
        userIDlookup[len(userIDlookup)] = u
    if not g in gameIDs: 
        gameIDs[g] = len(gameIDs)
        gameIDlookup[len(gameIDlookup)] = g

nUsers,nItems = len(userIDs),len(gameIDs)

for u, g, _ in hoursTrain:
    interactions.append((u, g, 1))

In [ ]:
class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(gameIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(gameIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb

    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaU) +\
                            tf.nn.l2_loss(self.gammaI))
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui

    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.1)
modelBPR = BPRbatch(3, 0.00001)

def trainingStepBPR(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ = [], [], []
        for _ in range(Nsamples):
            u,i,_ = random.choice(interactions) # positive sample
            j = random.choice(games) # negative sample
            while j in itemsPerUser[u]:
                j = random.choice(games)
            sampleU.append(userIDs[u])
            sampleI.append(gameIDs[i])
            sampleJ.append(gameIDs[j])

        loss = model(sampleU,sampleI,sampleJ)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [ ]:
for i in range(100):
    obj = trainingStepBPR(modelBPR, interactions)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

In [ ]:
pairsPlayed = {}
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        continue
    u,g = l.strip().split(',')
    if (u,g) not in pairsPlayed:
        pairsPlayed[(u,g)] = 0

predictions = []

# Initialize the list of predictions
predictions = []

gameCount = defaultdict(int)
total_played = 0

for user,game,_ in allHours:
  gameCount[game] += 1
  total_played += 1

# Calculate the median of the game counts
median_game_count = np.median(list(gameCount.values()))

user_games_test = defaultdict(list)
# For each user-game pair in the validation set
for user, game in pairsPlayed:
    if user in userIDs and game in gameIDs:
        score = modelBPR.predict(userIDs[user], gameIDs[game]).numpy()
        user_games_test[user].append((game, score))

predSet = {}
for user in user_games_test:
    user_games_test[user] = sorted(user_games_test[user], key=lambda x: x[1], reverse=True)

for user, game in pairsPlayed:
    if user not in userIDs and game in gameIDs:
        # Use dataset statistics or item features to make a prediction
        # For example, if the game is more popular than the median, predict 1, otherwise predict 0
        predSet[(user, game)] = 1 if gameCount[game] > median_game_count else 0
        continue
    elif game not in gameIDs or (user not in userIDs and game not in gameIDs):
        predSet[(user, game)] = 0
        continue
    games_t = user_games_test[user]
    if len(games_t) > 0:
        half = len(games_t) // 2
        # Add the first half of games_t as (user, game): 1
        for game, _ in games_t[:half]:
            predSet[(user, game)] = 1
        # Add the second half of games_t as (user, game): 0
        for game, _ in games_t[half:]:
            predSet[(user, game)] = 0
        
print(len(predSet))
print(predSet[("u25070741", "g03916495")])



In [ ]:
print(len(userIDs))

In [ ]:
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    pred = predSet[(u,g)]
    
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

## Time Played Prediction

In [103]:
userIDs = {}
gameIDs = {}
interactions = []

for u, g, r in allHours:
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not g in gameIDs: gameIDs[g] = len(gameIDs)
    h = r['hours_transformed']
    interactions.append((u,g,h))

random.shuffle(interactions)
len(interactions)

174999

In [104]:
nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

gamesPerUser = defaultdict(list)
usersPerGame = defaultdict(list)
for u,g,r in interactionsTrain:
    gamesPerUser[u].append(g)
    usersPerGame[g].append(u)



trainHours = [r[2]['hours_transformed'] for r in hoursTrain]
globalAverage = sum(trainHours) * 1.0 / len(trainHours)

hoursPerUser = defaultdict(list)
hoursPerItem = defaultdict(list)

hours_trans_list = []

for user, item, data in hoursTrain:
    hours = data['hours_transformed']
    hoursPerUser[user].append((item, hours))
    hoursPerItem[item].append((user, hours))
    hours_trans_list.append(hours)

betaU = {}
for u in hoursPerUser:
    betaU[u] = 0
betaI = {}
for g in hoursPerItem:
    betaI[g] = 0

mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)
print(mu)

trainHours = np.array(hours_trans_list)

# Calculate the median
ma = np.float32(np.median(hours_trans_list))
print(ma)

3.7162842372434857
3.4724877


In [105]:
optimizer = tf.keras.optimizers.legacy.Adam(0.1)
actual = [r for _, _, r in interactionsTest]

In [106]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, alp, K, lamb_beta, lamb_gamma, betaU_init=None, betaI_init=None):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(alp)
        # Initialize to small random values or use provided initial values
        if betaU_init is not None and isinstance(betaU_init, dict):
            betaU_values = [betaU_init.get(uid, tf.random.normal([], stddev=0.001)) for uid in userIDs]
            self.betaU = tf.Variable(betaU_values)
        else:
            self.betaU = tf.Variable(tf.random.normal([len(userIDs)], stddev=0.001))
        
        # Initialize betaI
        if betaI_init is not None and isinstance(betaI_init, dict):
            betaI_values = [betaI_init.get(iid, tf.random.normal([], stddev=0.001)) for iid in gameIDs]
            self.betaI = tf.Variable(betaI_values)
        else:
            self.betaI = tf.Variable(tf.random.normal([len(gameIDs)], stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(gameIDs),K],stddev=0.001))
        self.lamb_beta = lamb_beta
        self.lamb_gamma = lamb_gamma

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb_beta * (tf.reduce_sum(self.betaU**2) + tf.reduce_sum(self.betaI**2)) +\
               self.lamb_gamma * (tf.reduce_sum(self.gammaU**2) + tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)
    
def trainingStep(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,h = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(gameIDs[i])
            sampleR.append(h)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [107]:
def initialize_parameters(alpha_init=None, betaU_init=None, betaI_init=None):
    global alpha, betaU, betaI
    alpha = alpha_init if alpha_init is not None else ma
    if betaU_init is not None:
        betaU = betaU_init
    else:
        betaU = betaU
    if betaI_init is not None:
        betaI = betaI_init
    else:
        betaI = betaI
def iterate(lamb, alpha_init=None, betaU_init=None, betaI_init=None):
    global alpha, betaU, betaI
    alpha = alpha_init if alpha_init is not None else alpha
    betaU = betaU_init if betaU_init is not None else betaU
    betaI = betaI_init if betaI_init is not None else betaI

    iteration = 0
    while True:
        alpha_old = alpha
        betaU_old = betaU.copy()
        betaI_old = betaI.copy()

        alpha_sum = 0
        for user, item, hours in interactionsTrain:
            alpha_sum += hours - betaU[user] - betaI[item]

        for user in betaU:
            betaU[user] = sum([hours - alpha_old - betaI_old[game] for game, hours in hoursPerUser[user]]) / (lamb + len(hoursPerUser[user]))

        for item in betaI:
            betaI[item] = sum([hours - alpha_old - betaU[user] for user, hours in hoursPerItem[item]]) / (lamb + len(hoursPerItem[item]))

        alpha = alpha_sum / len(interactionsTrain)

        print(f"Iteration {iteration}:")
        print(f"Alpha difference: {abs(alpha - alpha_old)}")
        print(f"Max betaU difference: {max(abs(betaU[user] - betaU_old[user]) for user in betaU)}")
        print(f"Max betaI difference: {max(abs(betaI[item] - betaI_old[item]) for item in betaI)}")

        iteration += 1

        if abs(alpha - alpha_old) < 1e-4 and all(abs(betaU[user] - betaU_old[user]) < 1e-4 for user in betaU) and all(abs(betaI[item] - betaI_old[item]) < 1e-4 for item in betaI):
            break

In [108]:
initialize_parameters()
iterate(5)
validMSE = sum([(h - alpha - betaU[u] - betaI[i])**2 for u, i, h in interactionsTest]) / len(interactionsTest)
print("MSE on the validation set: ", validMSE)

Iteration 0:
Alpha difference: 0.24379654917891047
Max betaU difference: 4.615680528991672
Max betaI difference: 4.503527097744742
Iteration 1:
Alpha difference: 0.2741149420269968
Max betaU difference: 1.3101467790768297
Max betaI difference: 0.46566564539021993
Iteration 2:
Alpha difference: 0.24016368319012216
Max betaU difference: 0.4499507843388755
Max betaI difference: 0.10139875033879076
Iteration 3:
Alpha difference: 0.2710048998288963
Max betaU difference: 0.2960044149143068
Max betaI difference: 0.024810373929335938
Iteration 4:
Alpha difference: 0.2405830288731199
Max betaU difference: 0.25957176529340054
Max betaI difference: 0.06019234678461771
Iteration 5:
Alpha difference: 0.2676806799360065
Max betaU difference: 0.2811951131833209
Max betaI difference: 0.015333239086460004
Iteration 6:
Alpha difference: 0.2406403014336278
Max betaU difference: 0.25937873776478443
Max betaI difference: 0.051907550236099276
Iteration 7:
Alpha difference: 0.2647789582565965
Max betaU diffe

Iteration 25:
Alpha difference: 0.2459232034014165
Max betaU difference: 0.25257194155177776
Max betaI difference: 0.023554634075615866
Iteration 26:
Alpha difference: 0.23232687532834184
Max betaU difference: 0.2541550446939401
Max betaI difference: 0.03416276031438059
Iteration 27:
Alpha difference: 0.24413561454540922
Max betaU difference: 0.2510589811614614
Max betaI difference: 0.023712892533992502
Iteration 28:
Alpha difference: 0.23124746801816531
Max betaU difference: 0.25262980992643713
Max betaI difference: 0.03363955124979068
Iteration 29:
Alpha difference: 0.24237892151131168
Max betaU difference: 0.24957562259108326
Max betaI difference: 0.023835932748009714
Iteration 30:
Alpha difference: 0.23015546278967447
Max betaU difference: 0.2510976279455507
Max betaI difference: 0.033153667018244626
Iteration 31:
Alpha difference: 0.24065074466585168
Max betaU difference: 0.24811355230462817
Max betaI difference: 0.023932140450299733
Iteration 32:
Alpha difference: 0.2290528535518

In [109]:
iterate(5, alpha, betaU, betaI)
validMSE = sum([(h - alpha - betaU[u] - betaI[i])**2 for u, i, h in interactionsTest]) / len(interactionsTest)
print("MSE on the validation set: ", validMSE)

Iteration 0:
Alpha difference: 9.40473950423737e-05
Max betaU difference: 9.96848704246378e-05
Max betaI difference: 1.1287190881059672e-05
MSE on the validation set:  2.774729812467112


In [132]:
betaU_in = betaU.copy()
betaI_in = betaI.copy()
alpha_in = alpha

In [133]:
betaU_in

{'u70666506': 0.4443464772153395,
 'u18612571': -0.27618392278302456,
 'u34283088': -0.6820473723454694,
 'u16220374': 0.13209737850639292,
 'u01499286': 0.04150754201402006,
 'u73063505': 0.3032355984590459,
 'u29223775': 0.23750294792113433,
 'u44401308': 0.37053376153965817,
 'u45027672': -0.25748811219912215,
 'u33908704': 0.49682100118352085,
 'u27998358': -0.35579457647769214,
 'u36214177': -0.5828442377524022,
 'u73747744': 0.38847503405845557,
 'u97936673': -0.35976617156540563,
 'u25365202': 0.40691774716704315,
 'u08631099': -0.033379986553103445,
 'u52584928': -0.018309188057041743,
 'u09520763': 0.31676019943311645,
 'u04893836': -0.21374093466437788,
 'u58381940': -0.1833551055348079,
 'u85007552': -0.4732227899104498,
 'u48369340': -0.009298712673209432,
 'u79530461': -0.772564846753273,
 'u44157494': 0.5027265742657292,
 'u70118164': -0.9449716171394844,
 'u21352780': -0.09175565574248906,
 'u74354158': -0.1547340530727089,
 'u67890036': -0.8014497778728064,
 'u90421779'

In [135]:
betaI_in

{'g49368897': 0.31111334458402345,
 'g73495588': -0.9196190163013356,
 'g68047320': 1.4737556838901325,
 'g51234623': -0.8827866072127868,
 'g25723374': -0.6488955008424631,
 'g58025004': 0.7487954510394259,
 'g69033010': 1.4587454537239875,
 'g46446145': 1.7999478529517647,
 'g02903254': -0.10970123639020352,
 'g66086214': 0.14513439488630445,
 'g21544048': -0.5037911071972042,
 'g86787099': -1.025671354522241,
 'g23131507': -0.226307079952072,
 'g65055990': -0.5780771717576875,
 'g45124396': 1.3972974926064432,
 'g74380807': 0.33778086470737206,
 'g46738138': -0.33822689512116544,
 'g65420497': -0.34657571185424313,
 'g76381409': 1.0468911956446734,
 'g03420056': 0.11841823886195482,
 'g13492762': 0.11465812642831116,
 'g35678007': -1.132620017727678,
 'g28405264': -0.2005048499740479,
 'g60903743': -0.04233101987076477,
 'g92017077': 0.34067754481995793,
 'g98792097': 1.5564287032829642,
 'g11225866': 0.3776979640189113,
 'g02428379': -1.0622558119238341,
 'g39879078': 0.57036190698

In [136]:
alpha_in = np.float32(alpha_in)

alpha_in

3.1505675

In [57]:
delta = -.002  # Threshold for worsening MSE
n = 5  # Look-back window for MSE values
patience = 3  # Number of iterations to wait after early stopping condition is met
mse_values = [float('inf')] * n  # Initialize the list with infinity
patience_counter = 0  # Counter for patience after early stopping condition is met

# ma, 1, 0.000012, 0.00041

modelLFM1 = LatentFactorModel( mu, 1, 0.000017, 0.00052)

for i in range(200):
    trainingStep(modelLFM1, interactionsTrain)
    if (i % 10 == 9):
        predictions = [modelLFM1.predict(userIDs[u], gameIDs[i]).numpy() for u, i, _ in interactionsTest]
        mse = mean_squared_error(actual, predictions)
        print("iteration " + str(i+1) + ", MSE = " + str(mse))
        
        # Update the list of MSE values and check worsening condition
        mse_values.pop(0)
        mse_values.append(mse)

        # Check if the current MSE is worse than the best MSE in the look-back window by a certain threshold
        if i > 0 and all(mse > min(mse_values) for past_mse in mse_values):
            patience_counter += 1
            print(f"MSE worsening detected at iteration {i+1}. Patience counter: {patience_counter}")
            if patience_counter >= patience:
                print("Early stopping, MSE no longer improving")
                break
        else:
            patience_counter = 0  # Reset patience counter if MSE improves

iteration 10, MSE = 3.463911828150341
iteration 20, MSE = 3.287645912802592
iteration 30, MSE = 3.215046271055695
iteration 40, MSE = 3.157147403204874
iteration 50, MSE = 3.1414124818312663
iteration 60, MSE = 3.1421426084616413
MSE worsening detected at iteration 60. Patience counter: 1
iteration 70, MSE = 3.1350287493094218
iteration 80, MSE = 3.1299258525840488
iteration 90, MSE = 3.1159532120636575
iteration 100, MSE = 3.1269785293344268
MSE worsening detected at iteration 100. Patience counter: 1
iteration 110, MSE = 3.107720784373078
iteration 120, MSE = 3.111461856610214
MSE worsening detected at iteration 120. Patience counter: 1
iteration 130, MSE = 3.111505905613978
MSE worsening detected at iteration 130. Patience counter: 2
iteration 140, MSE = 3.11513682283635
MSE worsening detected at iteration 140. Patience counter: 3
Early stopping, MSE no longer improving


In [36]:
predictions = [modelLFM1.predict(userIDs[u], gameIDs[i]).numpy() for u, i, _ in interactionsTest]

mse = mean_squared_error(actual, predictions)

print("MSE: ", mse)

MSE:  3.045437828394217


In [ ]:
# u11400327,g33702735
modelLFM1.predict(userIDs['u11400327'], gameIDs['g33702735']).numpy()

In [28]:
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [29]:
interactions

[('u50508753', 'g66415217', 4.940166750482817),
 ('u50393013', 'g06097413', 3.3362833878644325),
 ('u31005947', 'g40499587', 8.087993146362317),
 ('u67703876', 'g51004348', 4.314696525656286),
 ('u95499376', 'g98214175', 6.03342300153745),
 ('u52414577', 'g61264440', 4.270528942380718),
 ('u61243657', 'g33211163', 2.786596361890807),
 ('u10803498', 'g05572229', 2.035623909730721),
 ('u70997740', 'g34777451', 3.6667565918848033),
 ('u75733314', 'g78518895', 2.807354922057604),
 ('u44742433', 'g71371078', 7.750874439656846),
 ('u05202684', 'g95302568', 0.48542682717024166),
 ('u49842694', 'g08020339', 4.095924419998536),
 ('u02338699', 'g49636917', 4.9818526532897405),
 ('u26892098', 'g42729032', 8.234098897837315),
 ('u41476260', 'g82275622', 1.8875252707415875),
 ('u34390978', 'g47036191', 4.498250867527825),
 ('u97802984', 'g92018877', 1.584962500721156),
 ('u84223749', 'g15714062', 7.6653359171851765),
 ('u91640743', 'g03207903', 0.6780719051126377),
 ('u91531964', 'g76381409', 5.485

In [32]:
# 0.000012, 0.00041
model = SVD(n_factors=1, reg_bu = 0.000012, reg_bi = 0.000012, reg_pu = 0.00041, reg_qi = 0.00041)

min_hours_transformed = 10000000
max_hours_transformed = 0
for u, g, h in allHours:
    hours = h['hours_transformed']
    min_hours_transformed = min(min_hours_transformed, hours)
    max_hours_transformed = max(max_hours_transformed, hours)

reader = Reader(rating_scale=(min_hours_transformed, max_hours_transformed))

# Convert the list to a DataFrame
interactions_df = pd.DataFrame(interactions, columns=['userID', 'gameID', 'hours_transformed'])


In [33]:
print(predictions[0].est)

sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

print(sse / len(predictions))

TypeError: '_io.TextIOWrapper' object is not subscriptable

In [35]:
from skopt import BayesSearchCV
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import PredefinedKFold
from surprise import Dataset, Reader, SVD
%pip install optuna
import optuna
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import cross_validate, KFold

min_hours_transformed = 10000000
max_hours_transformed = 0
for u, g, h in allHours:
    hours = h['hours_transformed']
    min_hours_transformed = min(min_hours_transformed, hours)
    max_hours_transformed = max(max_hours_transformed, hours)

def objective(trial):
    n_factors = trial.suggest_int('n_factors', 1, 10)
    n_epochs = trial.suggest_int('n_epochs', 1, 50)
    lr_all = trial.suggest_float('lr_all', 0.001, 0.1)
    reg_bu = trial.suggest_float('reg_bu', 0.000005, 0.001)
    reg_bi = trial.suggest_float('reg_bi', 0.000005, 0.001)
    reg_qi = trial.suggest_float('reg_qi', 0.00001, 0.01)
    reg_pu = trial.suggest_float('reg_pu', 0.00001, 0.01)

    svd = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_bu=reg_bu, reg_bi = reg_bi, reg_qi = reg_qi, reg_pu=reg_pu, biased=True)
    
    reader = Reader(rating_scale=(min_hours_transformed, max_hours_transformed))
    data = Dataset.load_from_df(interactions_df[['userID', 'gameID', 'hours_transformed']], reader)
    
    kf = KFold(n_splits=3)
    mse = []
    for trainset, testset in kf.split(data):
        svd.fit(trainset)
        predictions = svd.test(testset)
        mse_score = accuracy.mse(predictions, verbose=False)
        mse.append(mse_score)
    avg_mse = sum(mse) / len(mse)
    print(f"Average MSE for this trial: {avg_mse}")
    return avg_mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

print(study.best_params)

Note: you may need to restart the kernel to use updated packages.


[I 2023-11-20 23:08:16,274] A new study created in memory with name: no-name-264f8502-9f2a-47b5-9eac-b855c59ecdd8
[I 2023-11-20 23:08:19,027] Trial 0 finished with value: 3.837337550490073 and parameters: {'n_factors': 1, 'n_epochs': 48, 'lr_all': 0.040821145653096066, 'reg_bu': 0.0003265210562909992, 'reg_bi': 0.0003070659666855964, 'reg_qi': 0.0078561980207122, 'reg_pu': 0.00024193908045357237}. Best is trial 0 with value: 3.837337550490073.


Average MSE for this trial: 3.837337550490073


[I 2023-11-20 23:08:20,225] Trial 1 finished with value: 6.643194101813386 and parameters: {'n_factors': 9, 'n_epochs': 16, 'lr_all': 0.07540600698169396, 'reg_bu': 0.000936624311066824, 'reg_bi': 4.378979515533752e-05, 'reg_qi': 0.0023289486274241326, 'reg_pu': 0.003108229977841643}. Best is trial 0 with value: 3.837337550490073.


Average MSE for this trial: 6.643194101813386


...

Average MSE for this trial: 3.205649184703001


[I 2023-11-20 23:13:07,384] Trial 198 finished with value: 3.164486861824645 and parameters: {'n_factors': 1, 'n_epochs': 9, 'lr_all': 0.012711974937783047, 'reg_bu': 0.0006055255670528276, 'reg_bi': 0.0007536063796589279, 'reg_qi': 0.00459113352029066, 'reg_pu': 0.005854620490787274}. Best is trial 141 with value: 3.0982544669945025.


Average MSE for this trial: 3.164486861824645


[I 2023-11-20 23:13:08,439] Trial 199 finished with value: 3.440871072555859 and parameters: {'n_factors': 1, 'n_epochs': 11, 'lr_all': 0.026406757762507713, 'reg_bu': 0.00051483092451853, 'reg_bi': 0.0005638341295491236, 'reg_qi': 0.003758056826476465, 'reg_pu': 0.0038210146840534772}. Best is trial 141 with value: 3.0982544669945025.


Average MSE for this trial: 3.440871072555859
{'n_factors': 1, 'n_epochs': 11, 'lr_all': 0.006658144733850633, 'reg_bu': 0.0005138892494881303, 'reg_bi': 0.0007284583020828921, 'reg_qi': 0.005616393859849432, 'reg_pu': 0.006172457884462173}


In [37]:
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    pred = modelLFM1.predict(userIDs[u], gameIDs[g]).numpy()
    
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

In [110]:
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    bu = betaU.get(u, 0)
    bi = betaI.get(g, 0)
    
    _ = predictions.write(u + ',' + g + ',' + str(alpha + bu + bi) + '\n')

predictions.close()